# N Queens Problem

Brett Deaton - Fall 2021

A sandbox to play around with the n queens problem. See [OEIS A000170](https://oeis.org/A000170).

My algorithm, imported below, lives in [n_queens.py](../DailyCodingProb/n_queens.py).
Note, I have since migrated this work to its own package,
[queens](https://github.com/mbdeaton/queens).

### Setup

In [ ]:
# This is a hack, and the n_queens module should be replaced with a package
import sys
sys.path.append('../DailyCodingProb')

In [ ]:
from n_queens import NQueensConfig

### Tests

In [ ]:
n = 10
nq = NQueensConfig(n)

In [ ]:
print(nq.num_nonattacking_configs(1), "total nonattacking configurations for", n, "queens")

In [ ]:
nq.set_config([n//2]*n)
nq.display_config()
print(nq.num_nonattacking_configs(1), "nonattacking configurations forward of the setting")

In [ ]:
print("Following config is valid?", nq.find_next_nonattacking_config())
nq.display_config()

In [ ]:
nq.find_next_random_config()
print("random config")
nq.display_config()
print()
print("next valid config")
nq.find_next_nonattacking_config()
nq.display_config()

In [ ]:
print("Total number of valid configurations for n queens: n=[0,1,2,...]")
for i in range(12):
    print(NQueensConfig(i).num_nonattacking_configs(), end=" ")